# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [6]:
! git clone https://github.com/adragneel/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .



fatal: destination path 'exam_2025' already exists and is not an empty directory.


In [1]:
import copy
import numpy as np


import torch

**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [2]:
mySeed = 113

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [3]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.




  Pour estimer les coefficients $\theta_k$
  d'un modèle linéaire, la méthode la plus simple est l'estimation par moindres carrés ordinaires (OLS). Cette méthode minimise la somme des carrés des écarts entre les prédictions du modèle et les valeurs observées des cibles.




In [4]:
import numpy as np

# Assuming train_set and test_set are defined as in the provided code.
X = np.hstack((np.ones((train_set['inputs'].shape[0], 1)), train_set['inputs']))
y = train_set['targets']

# Use numpy's least squares solver to estimate theta
theta = np.linalg.lstsq(X, y, rcond=None)[0]

print("Estimated coefficients (theta):", theta)

Estimated coefficients (theta): [5.52416753 1.18625885 1.1083114  2.36999209]


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

Pour estimer les $\theta_k$
à l'aide d'un réseau de neurones entraîné par SGD, une architecture simple et efficace est un réseau de neurones linéaire à une seule couche car Un réseau linéaire sans activation est suffisant pour modéliser une relation linéaire simple  donc ajouter des couches ou des activations augmenterait inutilement la complexité sans gain en expressivité.
La simplicité du modèle (absence de surparamétrisation) limite le risque de surapprentissage.

In [7]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)
import torch.nn as nn

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(3, 1)  # Input features: 3 (x, y, z); Output features: 1 (t)

    def forward(self, x):
        return self.linear(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [8]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
      # Zero the gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = mySimpleNet(batch_inputs)
      loss = criterion(outputs.squeeze(), batch_targets)

      # Backward pass and optimization
      loss.backward()
      optimizer.step()
    if (epoch+1) % 100 == 0 :
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [100/500], Loss: 4.2323
Epoch [200/500], Loss: 3.7900
Epoch [300/500], Loss: 3.8637
Epoch [400/500], Loss: 3.2753
Epoch [500/500], Loss: 3.3045


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [10]:

# The estimated theta_k values are stored in the weights and bias of the linear layer within mySimpleNet.
for name, param in mySimpleNet.named_parameters():
    if param.requires_grad:
        print(name, param.data)

linear.weight tensor([[1.1876, 1.1081, 2.3712]])
linear.bias tensor([5.5253])


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [12]:

# Use the trained model to make predictions on the test set
X_test = torch.tensor(test_set['inputs'], dtype=torch.float32)
y_test_pred = mySimpleNet(X_test).detach().numpy()

# Assuming 'theta' from Q1 is still in scope
X_test_q1 = np.hstack((np.ones((test_set['inputs'].shape[0], 1)), test_set['inputs']))
y_test_pred_q1 = np.dot(X_test_q1, theta)

# Compare predictions
print("Predictions from neural network:", y_test_pred.flatten()[:5])
print("Predictions from OLS:", y_test_pred_q1[:5])

# Calculate and print Mean Squared Error (MSE) for both methods.
mse_nn = np.mean(np.square(y_test_pred.flatten() - test_set['targets']))
mse_ols = np.mean(np.square(y_test_pred_q1 - test_set['targets']))

print(f"Mean Squared Error (Neural Network): {mse_nn}")
print(f"Mean Squared Error (OLS): {mse_ols}")


Predictions from neural network: [4.077937  4.392635  6.6639805 8.663747  6.131276 ]
Predictions from OLS: [4.07862174 4.39219757 6.66042811 8.66222518 6.12840044]
Mean Squared Error (Neural Network): 3.7761703574449137
Mean Squared Error (OLS): 3.775926884486523


"L'erreur quadratique moyenne obtenue avec le réseau de neurones est proche de celle obtenue par la méthode analytique
 Cela montre que le réseau a correctement appris la relation linéaire sous-jacente, bien que les deux approches diffèrent dans leur nature.

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [18]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

Il s'agit d'un réseau de neurones convolutionnel causal (Causal Convolutional Neural Network, ou causalCNN).

In [23]:

# Nombre de paramètres dans self.Down1:
# La couche Down_causal effectue une convolution 1D suivie d'un max pooling.
# Convolution:  in_channels * out_channels * kernel_size + out_channels (biais)
# Ici, in_channels = 64, out_channels = 128, kernel_size = 3
# Donc, nombre de paramètres de la convolution = 64 * 128 * 3 + 128 = 24704



# Nombre de paramètres au total:
total_params = sum(p.numel() for p in model.parameters())
print(f"Nombre total de paramètres du modèle : {total_params}")

Nombre total de paramètres du modèle : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

In [ ]:

# The input size is reduced by the Down_causal layers, which consist of convolutional and max pooling layers.
# The convolutional layers reduce the number of features through convolution operation.
# The max pooling layers reduce the spatial dimension of the feature maps by downsampling the input.

# The size is restored in the second part of the network (the Up_causal layers) primarily using transposed convolutions (also known as deconvolutions).
# These operations upsample the feature maps, increasing their spatial dimensions.
# Additionally, skip connections from corresponding downsampling layers are often concatenated to the upsampled features.
# This helps recover finer details lost during downsampling, improving the quality of the upsampled feature maps.


**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

In [ ]:

# Le champ réceptif est augmenté par plusieurs mécanismes dans ce réseau :

# 1. Convolutions avec des noyaux de taille supérieure à 1 : Chaque convolution élargit le champ réceptif.
# 2. Dilatation : L'utilisation de la dilatation dans les convolutions permet d'élargir le champ réceptif sans augmenter le nombre de paramètres.
# 3. Couches de pooling : Les couches de pooling (max-pooling dans ce cas) réduisent la taille de la sortie, mais augmentent le champ réceptif des neurones de la couche suivante.

# Calcul de la taille du champ réceptif en sortie de self.inc :
# La couche self.inc est composée de deux convolutions 1D avec une dilation de 1 et un noyau de taille 3.

# Première convolution :
# Champ réceptif = taille du noyau = 3

# Deuxième convolution :
# Champ réceptif = champ réceptif précédent + (taille du noyau - 1) * dilation
# Champ réceptif = 3 + (3-1) * 1 = 5

# Donc, le champ réceptif en sortie de self.inc est de 5.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [21]:

input_tensor1 = torch.rand(1, 1, 10000)  # Séries temporelles d'entrée

# Calcul de la sortie pour l'entrée initiale
output_original = model(input_tensor1)

# Choisir un indice pour tester l'impact (ici on prend l'indice 5000)
index = 5000

# Créer une copie de l'entrée et modifier légèrement l'élément à l'indice choisi
input_tensor_modified = input_tensor1.clone()
input_tensor_modified[0, 0, index] += 1e-5  # Modification d'une petite valeur

# Calculer la sortie associée à l'entrée modifiée
output_modified = model(input_tensor_modified)

# Calcul de la différence entre les sorties pour déterminer l'impact
output_diff = torch.abs(output_modified - output_original)

# Afficher la différence et déterminer l'indice de la sortie affectée
print(f"Differences at index {index} in output:", output_diff[0, 0, index])

# Calcul de la taille du champ réceptif
receptive_field_size = torch.sum(output_diff[0, 0, index:] > 1e-5)  # Nombre d'éléments affectés dans la sortie
print(f"Estimated receptive field size for output at index {index}: {receptive_field_size.item()}")


Differences at index 5000 in output: tensor(6.7800e-07, grad_fn=<SelectBackward0>)
Estimated receptive field size for output at index 5000: 3


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



La difference entre output_original et output_modified est très faible.
Cela confirmerait que le réseau respecte la causalité, et que le modèle ne "regarde" pas les éléments futurs dans la série temporelle pour prédire y5000.

\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

In [ ]:

# Dans le contexte de l'apprentissage du classement (learning to rank), les "positive samples" et "negative samples" font référence à des paires d'éléments à classer.

# Positive samples: Une paire d'éléments où l'élément positif est considéré comme plus pertinent que l'élément négatif.
# Exemple: Dans un moteur de recherche, si une requête donne lieu à deux documents, le document le plus pertinent serait le "positive sample", tandis que le moins pertinent serait le "negative sample".



**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

In [ ]:

# Dans l'expression de la perte LRankNet, les zi représentent les scores de pertinence prédits par le modèle pour chaque document i.
# Ils sont produits par le modèle de classement, qui prend en entrée les caractéristiques des documents et renvoie un score de pertinence pour chaque document.
# Plus le score zi est élevé, plus le document i est considéré comme pertinent.

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?



The RankNet loss function aims to minimize the difference between the predicted scores of positive and negative samples.  Specifically, the loss function applies a sigmoid to the difference of the scores of a positive sample ($z_i$) and a negative sample ($z_j$).  The sigmoid function outputs a value between 0 and 1.

Let's analyze the loss function:

* **When the model correctly ranks the positive sample higher than the negative sample**:  $z_i$ > $z_j$. This leads to a positive input for the sigmoid function. As a result, the sigmoid's output is close to 1. The loss is then close to 0. The model is rewarded.

* **When the model incorrectly ranks the negative sample higher than the positive sample**: $z_j$ > $z_i$. This leads to a negative input for the sigmoid function.  The sigmoid's output approaches 0.  The loss function then approaches a value close to log(1) ≈ 0. This implies that the cost associated with this situation is high. Therefore, the model is penalized.

* **When the model predicts the same score for both samples ($z_i = z_j$):**  The sigmoid's input is 0, its output is 0.5, and the loss is non-zero.

During training, the optimizer adjusts the model's parameters to minimize the overall loss across all sample pairs.  This process essentially pushes the model to assign higher scores ($z_i$) to positive samples and lower scores ($z_j$) to negative samples, ensuring that the difference $z_i - z_j$ is positive, thereby minimizing the loss. This leads to the "estimated value of positive samples is greater than that of negative samples for each pair".  Therefore, the model learns to correctly rank samples, favoring the positive ones over the negative ones.


**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?

In [ ]:

# Dans le cadre d'une approche par deep learning, on utilise les termes suivants :

# Pour qualifier les réseaux de neurones exploités :

# - Réseau de neurones convolutifs (CNN) :  spécialement adaptés au traitement des données spatiales (images, vidéos) ou temporelles (signaux, séries temporelles).
# - Réseau de neurones récurrents (RNN) : conçus pour traiter des séquences de données, en tenant compte de la dépendance temporelle entre les éléments.
# - Réseaux de neurones à transformers (Transformer networks) : une architecture plus récente, très performante pour les tâches de traitement du langage naturel et de plus en plus utilisée pour d'autres types de données.
# - Réseau de neurones denses (Fully Connected Network, FCN) : chaque neurone est connecté à tous les neurones de la couche précédente.
# - Auto-encodeurs : utilisés pour l'apprentissage non supervisé, afin de réduire la dimensionnalité des données ou de détecter des anomalies.
# - Générateurs adversaires (GAN) : deux réseaux antagonistes qui apprennent à générer de nouvelles données similaires aux données d'apprentissage.

# Pour la modalité suivant laquelle ils sont entraînés :

# - Apprentissage supervisé : le modèle est entraîné sur des données étiquetées, c'est-à-dire que chaque donnée est associée à une cible (label).
# - Apprentissage non supervisé : le modèle est entraîné sur des données non étiquetées, afin de découvrir des structures ou des motifs cachés.
# - Apprentissage par renforcement : le modèle apprend à interagir avec un environnement, en recevant des récompenses ou des pénalités selon ses actions.
# - Apprentissage semi-supervisé : le modèle est entraîné sur un ensemble de données étiquetées et non étiquetées.
# - Apprentissage par transfert : utiliser un modèle pré-entraîné sur un grand ensemble de données pour une tâche spécifique et l'adapter pour une tâche différente.